In [ ]:
# install apt dependencies
!sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
!wget -qO- https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo tee /etc/apt/trusted.gpg.d/pgdg.asc &>/dev/null
!sudo apt update
!sudo apt install postgresql postgresql-client postgresql-server-dev-all postgresql-16-pgvector

In [ ]:
# setup database 
!service postgresql start

!sudo -u postgres psql -c "CREATE USER emotion_classification WITH SUPERUSER PASSWORD 'password';"
!echo "127.0.0.1:5432:emotion_classification:emotion_classification:password" > ~/.pgpass
!chmod 0600 ~/.pgpass

!sudo -u postgres psql -c "CREATE EXTENSION vector;"
!sudo -u postgres createdb emotion_classification

# TODO: specify URL
!wget -O emotion_classification.dump "ENTER URL HERE"

In [ ]:
# import database from dump
!sudo pg_restore -U emotion_classification -d emotion_classification -h 127.0.0.1 --jobs=8 emotion_classification.dump

In [ ]:
# clone the Barometrs repository
!git clone https://github.com/LUMII-AILab/Barometrs.git
%cd Barometrs

In [ ]:
# install Python dependencies
!pip install -r requirements.dev.txt

In [ ]:
# execute comment prediction script
%run google_colab/predict_comments_script.py

In [ ]:
# export database to dump
!sudo pg_dump -U emotion_classification -d emotion_classification -h 127.0.0.1 -Fc -Z 9 -f /tmp/emotion_classification.dump

# download dump
from google.colab import files
files.download('/tmp/emotion_classification.dump')

In [ ]:
# terminate the Google Colab instance
from google.colab import runtime
runtime.unassign()